In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD
from keras.datasets import mnist

from tqdm import tqdm
import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def Generator():
    
    model = Sequential()
    model.add(Dense(128, input_dim=100))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(28*28))
    model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    
    return model

In [3]:
def Discriminator():
        
    model = Sequential()
    model.add(Dense(512, input_dim=784))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(units=1))
    model.add(Activation('sigmoid'))
    
    optimizer = SGD(lr=0.1, momentum=0.3, decay=1e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return model

In [4]:
def GAN(generator, discriminator):

    discriminator.trainable = False

    model = Sequential()
    
    model.add(generator)
    model.add(discriminator)
    
    optimizer = SGD(lr=0.1, momentum=0.3, decay=1e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return model

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 28*28)
X_test = X_test.reshape(X_test.shape[0], 28*28)
X_train.astype('float32')
X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255

print('X_train shape', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape (60000, 784)
60000 train samples
10000 test samples


In [6]:
generator = Generator()
discriminator = Discriminator()
gan = GAN(generator, discriminator)

epochs = 30
batch_size = 32
input_size = 100

num_batches = int(X_train.shape[0] / batch_size)

pbar = tqdm(total=epochs * num_batches)

gan_loss = []
discriminator_loss = []

for epoch in range(epochs):
    
    for index in range(num_batches):

        pbar.update(1)

        # Generative data
        noise = np.random.uniform(0, 1, size=[batch_size, input_size])
        generated_data = generator.predict_on_batch(noise)

        # Training data chosen from Mnist samples
        training_data = X_train[index * batch_size: (index + 1) * batch_size]

        X = np.vstack((generated_data, training_data))
        y = np.zeros(2 * batch_size)
        y[:batch_size] = 1

        # Train discriminator
        d_loss = discriminator.train_on_batch(x=X, y=y)

        # Train generator (Seemingly train GAN but the discriminator in the model is disabled to train.)
        noise = np.random.uniform(0, 1, size=[batch_size, input_size])
        y = np.zeros(batch_size)
        g_loss = gan.train_on_batch(x=noise, y=y)

        discriminator_loss.append(d_loss)
        gan_loss.append(g_loss)

    # Plot losses
    fig = plt.figure(figsize=(10, 5))        
    fig.suptitle('epoch: ' + str(epoch + 1))
    plt.plot(discriminator_loss, label="discriminator's loss", color='b')
    plt.plot(gan_loss, label="generator's loss", color='r')
    plt.xlim([0, epochs * num_batches])
    plt.legend()
    plt.savefig('./gan-loss/' + str(epoch + 1) + '.png')
    plt.close()        

    # Visualize generated data
    generated_images = generator.predict(noise)

    fig = plt.figure(figsize=(9, 9))        
    for i in range(9):
        plt.subplot(3, 3, i+1)
        img = generated_images[i, :]
        img = img.reshape((28, 28))
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
        plt.axis('off')
    plt.savefig('./gan-images/' + str(epoch + 1) + '.png')
    plt.close()     
        
pbar.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56250/56250 [17:05<00:00, 54.87it/s]
